# Question 13: Numerical derivatives

## 13(a) Central Difference Scheme

In [15]:
# Adaoted from the code provided in Lecture Unit 4.

using Plots
using LaTeXStrings

function calculate_numerical_derivative_error(
        x0, f, fprime;
        h_range = 10 .^(-14:0.01:-2),
        diff_scheme = (f, x; h=sqrt(eps())) -> (f(x + h) - f(x)) / h
    )
    errs = [abs(diff_scheme(f, x0; h = h) - fprime(x0))/abs(fprime(x0)) for h in h_range]
    return errs
end

function plot_numerical_derivative_error(x0, f, fprime;
    h_range = 10 .^(-14:0.01:-2),
    diff_scheme = (f, x; h=sqrt(eps())) -> (f(x + h) - f(x)) / h,
    colour = :red,
    label = ""
)
    errs = calculate_numerical_derivative_error(x0, f, fprime; h_range=h_range, diff_scheme=diff_scheme)
    plot(h_range, errs,
        yaxis = :log,xaxis = :log,
        xlabel="h" ,ylabel="Absolute relative error",
        c = colour,
        label=label,
        legend = :bottomleft,
        size=(800, 600),
        title = L"\mathrm{I\ hope\ you\ don't\ have\ colour\ blindness}",
        xticks=10,
        yticks=10,
    )
end

function plot_numerical_derivative_error!(x0, f, fprime;
    h_range = 10 .^(-14:0.01:-2),
    diff_scheme = (f, x; h=sqrt(eps())) -> (f(x + h) - f(x)) / h,
    colour = :red,
    label = ""
)
    errs = calculate_numerical_derivative_error(x0, f, fprime; h_range=h_range, diff_scheme=diff_scheme)
    plot!(h_range, errs,
        c = colour,
        label=label
    )
end

diff_forward(f, x; h = sqrt(eps())) = (f(x + h) - f(x)) / h
diff_central(f, x; h = sqrt(eps())) = (f(x + h/2) - f(x - h/2)) / h

plot_numerical_derivative_error(0.5, x -> sin(x^2), x -> 2x*cos(x^2); diff_scheme=diff_central, colour="#4876c7", label=L"\mathrm{(Cen.)}\ f_1(x) = \sin\left(x^2\right)")
plot_numerical_derivative_error!(0.5, x -> sin(x^2), x -> 2x*cos(x^2); diff_scheme=diff_forward, colour="#30c26b", label=L"\mathrm{(For.)}\ f_1(x) = \sin\left(x^2\right)")

plot_numerical_derivative_error!(1, x -> exp(3*x^2), x -> 6x * exp(3*x^2); diff_scheme=diff_central, colour="#1d4182", label=L"\mathrm{(Cen.)}\ f_2(x) = \exp\left(3x^2\right)")
plot_numerical_derivative_error!(1, x -> exp(3*x^2), x -> 6x * exp(3*x^2); diff_scheme=diff_forward, colour="#3c9e64", label=L"\mathrm{(For.)}\ f_2(x) = \exp\left(3x^2\right)")

f3 = x -> atan(2x) / (1 + exp(-2*x^2))
f3prime = x -> (2*exp(2*x^2)*(1 + exp(2*x^2) + 2*(x + 4*x^3)*atan(2*x))) / ((1 + exp(2*x^2))^2*(1 + 4*x^2))
plot_numerical_derivative_error!(2, f3, f3prime; diff_scheme=diff_central, colour="#648dd9", label=L"\mathrm{(Cen.)}\ f_3(x) = \arctan\left(2x\right) / \left(1 + \exp\left(-2x^2\right)\right)")
plot_numerical_derivative_error!(2, f3, f3prime; diff_scheme=diff_forward, colour=:"#0f6331", label=L"\mathrm{(For.)}\ f_3(x) = \arctan\left(2x\right) / \left(1 + \exp\left(-2x^2\right)\right)")

critical_h = 10^(-5.6)
plot!([critical_h, critical_h], [eps(), 10e0], lw=4, label=L"\mathrm{Critical}~ h~ \mathrm{value}")

savefig("./figure.png")

"c:\\Users\\s4430266\\Julia-Codespaces\\dev\\figure.png"

## 13(b) Softmax Evaluations of Vector Function Jacobian with Forwards vs Central Difference Scheme

The forward difference scheme needs $K+1$ `softmax` evaluations, while the central difference scheme requires $2K$. This is because each softmax call returns all $K$ output components at once, so one call per perturbed coordinate gives a whole column of the Jacobian. The forward difference calls it once at a time, while the central difference scheme calls it twice.

## 13(c) Error of Jacobian using Forwards vs Central Difference Schemes